In [1]:
import os
import numpy as np
import music21 as m21
import pandas as pd
import json
import matplotlib.pyplot as plt
from scipy import stats
from scipy import spatial
import time
import math
from IPython.display import display
from collections import Counter

np.random.seed(777)

## Functions

In [2]:
# Function to retrieve a list of midi pitch events and its timestamp
def getMelodyDeltaTimes(eventsintrack):
    
    # Initialize array
    DeltaTimes = []
    
    # Initialize cumulative sum
    cum_sum = 0
    
    # Initialize variable to track the time delta
    prev_deltatime = 0
    
    # Traverse the events
    for ev in eventsintrack:
        
        # If a note starts
        if (ev.isNoteOn()):
            
            # Get the pitch name and save it with the cumulative sum, midi pitch and name
            pitch_in_time = m21.pitch.Pitch(ev.pitch)
            DeltaTimes.append((cum_sum, prev_deltatime, pitch_in_time.midi, pitch_in_time.spanish, pitch_in_time))
            
            # Restart the delta time
            prev_deltatime = 0
        
        # Else if there is a delta time
        elif(str(ev.type) == "DeltaTime"):
            
            # We sum the time
            cum_sum += ev.time
            
            # We sum it to the current delta time
            prev_deltatime += ev.time
    
    # Return the array
    return DeltaTimes

In [3]:
def get_MelodyShapeNgram_NOREST(melody_w_times):
    ngram_list = []
    for m_el in melody_w_times:
        # print(m_el)
        current_element = [m_el[2], m_el[0], max(m_el[1],1),  0]
        # print(current_element)
        ngram_list.append(current_element)
    return ngram_list

# Groups for analysis
 * Reference for the axis
 * Groups of dataset

In [4]:
Y_AXIS_MIDI_PATHS = [
    "./CalebRascon/CORPUS/MIDI/",
    "./CalebRascon/MIDI_Grammar_SOLO_LEN12/",
    "./MilesDavis/MIDI_Grammar_SOLO_LEN12/",
    "./CharlieParker/MIDI_Grammar_SOLO_LEN12/"
]

In [5]:
X_AXIS_MIDI_PATHS = [
    "./CalebRascon/CORPUS/MIDI/",
    "./CalebRascon/MIDI_Grammar_SOLO_LEN12/",
    "./MilesDavis/MIDI_Grammar_SOLO_LEN12/",
    "./CharlieParker/MIDI_Grammar_SOLO_LEN12/",
    "./CalebRascon/MIDI_Grammar_TRADE_Caleb/",
    "./CalebRascon/MIDI_Grammar_TRADE_Miles/",
    "./CalebRascon/MIDI_Grammar_TRADE_CharlieParker/"
]

In [6]:
%%time
note_representation_bspline = {"fam":{}}
for MIDI_path_query in Y_AXIS_MIDI_PATHS:
    for MIDI_path_test in X_AXIS_MIDI_PATHS:
        
        similarities_all_v_all = {}
        for root_ref, dirs_ref, files_ref in os.walk(MIDI_path_query):
            for name_ref in files_ref:
                # print("+++++++++++++++++++++++++++++")
                # print(name_ref)
                melody_score_A = m21.converter.parseFile(os.path.join(root_ref, name_ref))
                midi_tracks_A = m21.midi.translate.streamToMidiFile(melody_score_A)
                melody_w_times_A = getMelodyDeltaTimes(midi_tracks_A.tracks[0].events)

                similarities_from_reference = []
                similarities_all_v_all[name_ref] = {}
                for root, dirs, files in os.walk(MIDI_path_test):
                    for name in files:

                        # print(name)
                        melody_score_B = m21.converter.parseFile(os.path.join(root, name))
                        midi_tracks_B = m21.midi.translate.streamToMidiFile(melody_score_B)
                        melody_w_times_B = getMelodyDeltaTimes(midi_tracks_B.tracks[0].events)
                        
                        
                        comparison_id = "MP_A;{0};M_A;{1};MP_B;{2};M_B;{3}".format(MIDI_path_query, name_ref, MIDI_path_test, name)
                        print(comparison_id)
                        # We Save the representation for the family
                        note_representation_bspline["fam"][comparison_id] = {"msa":[], "msb":[]}
                        note_representation_bspline["fam"][comparison_id]["msa"] = get_MelodyShapeNgram_NOREST(melody_w_times_A)
                        note_representation_bspline["fam"][comparison_id]["msb"] = get_MelodyShapeNgram_NOREST(melody_w_times_B)

MP_A;./CalebRascon/CORPUS/MIDI/;M_A;rola3.mid;MP_B;./CalebRascon/CORPUS/MIDI/;M_B;rola3.mid
MP_A;./CalebRascon/CORPUS/MIDI/;M_A;rola3.mid;MP_B;./CalebRascon/CORPUS/MIDI/;M_B;rola2.mid
MP_A;./CalebRascon/CORPUS/MIDI/;M_A;rola3.mid;MP_B;./CalebRascon/CORPUS/MIDI/;M_B;rola4.mid
MP_A;./CalebRascon/CORPUS/MIDI/;M_A;rola3.mid;MP_B;./CalebRascon/CORPUS/MIDI/;M_B;rola5.mid
MP_A;./CalebRascon/CORPUS/MIDI/;M_A;rola3.mid;MP_B;./CalebRascon/CORPUS/MIDI/;M_B;rola1.mid
MP_A;./CalebRascon/CORPUS/MIDI/;M_A;rola2.mid;MP_B;./CalebRascon/CORPUS/MIDI/;M_B;rola3.mid
MP_A;./CalebRascon/CORPUS/MIDI/;M_A;rola2.mid;MP_B;./CalebRascon/CORPUS/MIDI/;M_B;rola2.mid
MP_A;./CalebRascon/CORPUS/MIDI/;M_A;rola2.mid;MP_B;./CalebRascon/CORPUS/MIDI/;M_B;rola4.mid
MP_A;./CalebRascon/CORPUS/MIDI/;M_A;rola2.mid;MP_B;./CalebRascon/CORPUS/MIDI/;M_B;rola5.mid
MP_A;./CalebRascon/CORPUS/MIDI/;M_A;rola2.mid;MP_B;./CalebRascon/CORPUS/MIDI/;M_B;rola1.mid
MP_A;./CalebRascon/CORPUS/MIDI/;M_A;rola4.mid;MP_B;./CalebRascon/CORPUS/MIDI/;M_

MP_A;./CalebRascon/CORPUS/MIDI/;M_A;rola1.mid;MP_B;./MilesDavis/MIDI_Grammar_SOLO_LEN12/;M_B;MilesDavis_Grammar_SOLO_LEN12_004.mid
MP_A;./CalebRascon/CORPUS/MIDI/;M_A;rola1.mid;MP_B;./MilesDavis/MIDI_Grammar_SOLO_LEN12/;M_B;MilesDavis_Grammar_SOLO_LEN12_005.mid
MP_A;./CalebRascon/CORPUS/MIDI/;M_A;rola1.mid;MP_B;./MilesDavis/MIDI_Grammar_SOLO_LEN12/;M_B;MilesDavis_Grammar_SOLO_LEN12_001.mid
MP_A;./CalebRascon/CORPUS/MIDI/;M_A;rola1.mid;MP_B;./MilesDavis/MIDI_Grammar_SOLO_LEN12/;M_B;MilesDavis_Grammar_SOLO_LEN12_003.mid
MP_A;./CalebRascon/CORPUS/MIDI/;M_A;rola1.mid;MP_B;./MilesDavis/MIDI_Grammar_SOLO_LEN12/;M_B;MilesDavis_Grammar_SOLO_LEN12_002.mid
MP_A;./CalebRascon/CORPUS/MIDI/;M_A;rola3.mid;MP_B;./CharlieParker/MIDI_Grammar_SOLO_LEN12/;M_B;CharlieParker_Grammar_SOLO_LEN12_002.mid
MP_A;./CalebRascon/CORPUS/MIDI/;M_A;rola3.mid;MP_B;./CharlieParker/MIDI_Grammar_SOLO_LEN12/;M_B;CharlieParker_Grammar_SOLO_LEN12_004.mid
MP_A;./CalebRascon/CORPUS/MIDI/;M_A;rola3.mid;MP_B;./CharlieParker/MIDI

MP_A;./CalebRascon/CORPUS/MIDI/;M_A;rola4.mid;MP_B;./CalebRascon/MIDI_Grammar_TRADE_CharlieParker/;M_B;trade_Caleb_CharlieParker_rola3.mid
MP_A;./CalebRascon/CORPUS/MIDI/;M_A;rola4.mid;MP_B;./CalebRascon/MIDI_Grammar_TRADE_CharlieParker/;M_B;trade_Caleb_CharlieParker_rola2.mid
MP_A;./CalebRascon/CORPUS/MIDI/;M_A;rola4.mid;MP_B;./CalebRascon/MIDI_Grammar_TRADE_CharlieParker/;M_B;trade_Caleb_CharlieParker_rola1.mid
MP_A;./CalebRascon/CORPUS/MIDI/;M_A;rola4.mid;MP_B;./CalebRascon/MIDI_Grammar_TRADE_CharlieParker/;M_B;trade_Caleb_CharlieParker_rola4.mid
MP_A;./CalebRascon/CORPUS/MIDI/;M_A;rola4.mid;MP_B;./CalebRascon/MIDI_Grammar_TRADE_CharlieParker/;M_B;trade_Caleb_CharlieParker_rola5.mid
MP_A;./CalebRascon/CORPUS/MIDI/;M_A;rola5.mid;MP_B;./CalebRascon/MIDI_Grammar_TRADE_CharlieParker/;M_B;trade_Caleb_CharlieParker_rola3.mid
MP_A;./CalebRascon/CORPUS/MIDI/;M_A;rola5.mid;MP_B;./CalebRascon/MIDI_Grammar_TRADE_CharlieParker/;M_B;trade_Caleb_CharlieParker_rola2.mid
MP_A;./CalebRascon/CORPUS/M

MP_A;./CalebRascon/MIDI_Grammar_SOLO_LEN12/;M_A;CalebRascon_Grammar_SOLO_LEN12_004.mid;MP_B;./CalebRascon/MIDI_Grammar_SOLO_LEN12/;M_B;CalebRascon_Grammar_SOLO_LEN12_005.mid
MP_A;./CalebRascon/MIDI_Grammar_SOLO_LEN12/;M_A;CalebRascon_Grammar_SOLO_LEN12_004.mid;MP_B;./CalebRascon/MIDI_Grammar_SOLO_LEN12/;M_B;CalebRascon_Grammar_SOLO_LEN12_004.mid
MP_A;./CalebRascon/MIDI_Grammar_SOLO_LEN12/;M_A;CalebRascon_Grammar_SOLO_LEN12_004.mid;MP_B;./CalebRascon/MIDI_Grammar_SOLO_LEN12/;M_B;CalebRascon_Grammar_SOLO_LEN12_002.mid
MP_A;./CalebRascon/MIDI_Grammar_SOLO_LEN12/;M_A;CalebRascon_Grammar_SOLO_LEN12_002.mid;MP_B;./CalebRascon/MIDI_Grammar_SOLO_LEN12/;M_B;CalebRascon_Grammar_SOLO_LEN12_003.mid
MP_A;./CalebRascon/MIDI_Grammar_SOLO_LEN12/;M_A;CalebRascon_Grammar_SOLO_LEN12_002.mid;MP_B;./CalebRascon/MIDI_Grammar_SOLO_LEN12/;M_B;CalebRascon_Grammar_SOLO_LEN12_001.mid
MP_A;./CalebRascon/MIDI_Grammar_SOLO_LEN12/;M_A;CalebRascon_Grammar_SOLO_LEN12_002.mid;MP_B;./CalebRascon/MIDI_Grammar_SOLO_LEN12/

MP_A;./CalebRascon/MIDI_Grammar_SOLO_LEN12/;M_A;CalebRascon_Grammar_SOLO_LEN12_004.mid;MP_B;./CharlieParker/MIDI_Grammar_SOLO_LEN12/;M_B;CharlieParker_Grammar_SOLO_LEN12_002.mid
MP_A;./CalebRascon/MIDI_Grammar_SOLO_LEN12/;M_A;CalebRascon_Grammar_SOLO_LEN12_004.mid;MP_B;./CharlieParker/MIDI_Grammar_SOLO_LEN12/;M_B;CharlieParker_Grammar_SOLO_LEN12_004.mid
MP_A;./CalebRascon/MIDI_Grammar_SOLO_LEN12/;M_A;CalebRascon_Grammar_SOLO_LEN12_004.mid;MP_B;./CharlieParker/MIDI_Grammar_SOLO_LEN12/;M_B;CharlieParker_Grammar_SOLO_LEN12_005.mid
MP_A;./CalebRascon/MIDI_Grammar_SOLO_LEN12/;M_A;CalebRascon_Grammar_SOLO_LEN12_004.mid;MP_B;./CharlieParker/MIDI_Grammar_SOLO_LEN12/;M_B;CharlieParker_Grammar_SOLO_LEN12_003.mid
MP_A;./CalebRascon/MIDI_Grammar_SOLO_LEN12/;M_A;CalebRascon_Grammar_SOLO_LEN12_004.mid;MP_B;./CharlieParker/MIDI_Grammar_SOLO_LEN12/;M_B;CharlieParker_Grammar_SOLO_LEN12_001.mid
MP_A;./CalebRascon/MIDI_Grammar_SOLO_LEN12/;M_A;CalebRascon_Grammar_SOLO_LEN12_002.mid;MP_B;./CharlieParker/MI

...

MP_A;./MilesDavis/MIDI_Grammar_SOLO_LEN12/;M_A;MilesDavis_Grammar_SOLO_LEN12_003.mid;MP_B;./CharlieParker/MIDI_Grammar_SOLO_LEN12/;M_B;CharlieParker_Grammar_SOLO_LEN12_003.mid
MP_A;./MilesDavis/MIDI_Grammar_SOLO_LEN12/;M_A;MilesDavis_Grammar_SOLO_LEN12_003.mid;MP_B;./CharlieParker/MIDI_Grammar_SOLO_LEN12/;M_B;CharlieParker_Grammar_SOLO_LEN12_001.mid
MP_A;./MilesDavis/MIDI_Grammar_SOLO_LEN12/;M_A;MilesDavis_Grammar_SOLO_LEN12_002.mid;MP_B;./CharlieParker/MIDI_Grammar_SOLO_LEN12/;M_B;CharlieParker_Grammar_SOLO_LEN12_002.mid
MP_A;./MilesDavis/MIDI_Grammar_SOLO_LEN12/;M_A;MilesDavis_Grammar_SOLO_LEN12_002.mid;MP_B;./CharlieParker/MIDI_Grammar_SOLO_LEN12/;M_B;CharlieParker_Grammar_SOLO_LEN12_004.mid
MP_A;./MilesDavis/MIDI_Grammar_SOLO_LEN12/;M_A;MilesDavis_Grammar_SOLO_LEN12_002.mid;MP_B;./CharlieParker/MIDI_Grammar_SOLO_LEN12/;M_B;CharlieParker_Grammar_SOLO_LEN12_005.mid
MP_A;./MilesDavis/MIDI_Grammar_SOLO_LEN12/;M_A;MilesDavis_Grammar_SOLO_LEN12_002.mid;MP_B;./CharlieParker/MIDI_Grammar_S

MP_A;./MilesDavis/MIDI_Grammar_SOLO_LEN12/;M_A;MilesDavis_Grammar_SOLO_LEN12_003.mid;MP_B;./CalebRascon/MIDI_Grammar_TRADE_CharlieParker/;M_B;trade_Caleb_CharlieParker_rola1.mid
MP_A;./MilesDavis/MIDI_Grammar_SOLO_LEN12/;M_A;MilesDavis_Grammar_SOLO_LEN12_003.mid;MP_B;./CalebRascon/MIDI_Grammar_TRADE_CharlieParker/;M_B;trade_Caleb_CharlieParker_rola4.mid
MP_A;./MilesDavis/MIDI_Grammar_SOLO_LEN12/;M_A;MilesDavis_Grammar_SOLO_LEN12_003.mid;MP_B;./CalebRascon/MIDI_Grammar_TRADE_CharlieParker/;M_B;trade_Caleb_CharlieParker_rola5.mid
MP_A;./MilesDavis/MIDI_Grammar_SOLO_LEN12/;M_A;MilesDavis_Grammar_SOLO_LEN12_002.mid;MP_B;./CalebRascon/MIDI_Grammar_TRADE_CharlieParker/;M_B;trade_Caleb_CharlieParker_rola3.mid
MP_A;./MilesDavis/MIDI_Grammar_SOLO_LEN12/;M_A;MilesDavis_Grammar_SOLO_LEN12_002.mid;MP_B;./CalebRascon/MIDI_Grammar_TRADE_CharlieParker/;M_B;trade_Caleb_CharlieParker_rola2.mid
MP_A;./MilesDavis/MIDI_Grammar_SOLO_LEN12/;M_A;MilesDavis_Grammar_SOLO_LEN12_002.mid;MP_B;./CalebRascon/MIDI_G

MP_A;./CharlieParker/MIDI_Grammar_SOLO_LEN12/;M_A;CharlieParker_Grammar_SOLO_LEN12_003.mid;MP_B;./CalebRascon/MIDI_Grammar_SOLO_LEN12/;M_B;CalebRascon_Grammar_SOLO_LEN12_002.mid
MP_A;./CharlieParker/MIDI_Grammar_SOLO_LEN12/;M_A;CharlieParker_Grammar_SOLO_LEN12_001.mid;MP_B;./CalebRascon/MIDI_Grammar_SOLO_LEN12/;M_B;CalebRascon_Grammar_SOLO_LEN12_003.mid
MP_A;./CharlieParker/MIDI_Grammar_SOLO_LEN12/;M_A;CharlieParker_Grammar_SOLO_LEN12_001.mid;MP_B;./CalebRascon/MIDI_Grammar_SOLO_LEN12/;M_B;CalebRascon_Grammar_SOLO_LEN12_001.mid
MP_A;./CharlieParker/MIDI_Grammar_SOLO_LEN12/;M_A;CharlieParker_Grammar_SOLO_LEN12_001.mid;MP_B;./CalebRascon/MIDI_Grammar_SOLO_LEN12/;M_B;CalebRascon_Grammar_SOLO_LEN12_005.mid
MP_A;./CharlieParker/MIDI_Grammar_SOLO_LEN12/;M_A;CharlieParker_Grammar_SOLO_LEN12_001.mid;MP_B;./CalebRascon/MIDI_Grammar_SOLO_LEN12/;M_B;CalebRascon_Grammar_SOLO_LEN12_004.mid
MP_A;./CharlieParker/MIDI_Grammar_SOLO_LEN12/;M_A;CharlieParker_Grammar_SOLO_LEN12_001.mid;MP_B;./CalebRascon/

MP_A;./CharlieParker/MIDI_Grammar_SOLO_LEN12/;M_A;CharlieParker_Grammar_SOLO_LEN12_003.mid;MP_B;./CharlieParker/MIDI_Grammar_SOLO_LEN12/;M_B;CharlieParker_Grammar_SOLO_LEN12_002.mid
MP_A;./CharlieParker/MIDI_Grammar_SOLO_LEN12/;M_A;CharlieParker_Grammar_SOLO_LEN12_003.mid;MP_B;./CharlieParker/MIDI_Grammar_SOLO_LEN12/;M_B;CharlieParker_Grammar_SOLO_LEN12_004.mid
MP_A;./CharlieParker/MIDI_Grammar_SOLO_LEN12/;M_A;CharlieParker_Grammar_SOLO_LEN12_003.mid;MP_B;./CharlieParker/MIDI_Grammar_SOLO_LEN12/;M_B;CharlieParker_Grammar_SOLO_LEN12_005.mid
MP_A;./CharlieParker/MIDI_Grammar_SOLO_LEN12/;M_A;CharlieParker_Grammar_SOLO_LEN12_003.mid;MP_B;./CharlieParker/MIDI_Grammar_SOLO_LEN12/;M_B;CharlieParker_Grammar_SOLO_LEN12_003.mid
MP_A;./CharlieParker/MIDI_Grammar_SOLO_LEN12/;M_A;CharlieParker_Grammar_SOLO_LEN12_003.mid;MP_B;./CharlieParker/MIDI_Grammar_SOLO_LEN12/;M_B;CharlieParker_Grammar_SOLO_LEN12_001.mid
MP_A;./CharlieParker/MIDI_Grammar_SOLO_LEN12/;M_A;CharlieParker_Grammar_SOLO_LEN12_001.mid

MP_A;./CharlieParker/MIDI_Grammar_SOLO_LEN12/;M_A;CharlieParker_Grammar_SOLO_LEN12_005.mid;MP_B;./CalebRascon/MIDI_Grammar_TRADE_CharlieParker/;M_B;trade_Caleb_CharlieParker_rola1.mid
MP_A;./CharlieParker/MIDI_Grammar_SOLO_LEN12/;M_A;CharlieParker_Grammar_SOLO_LEN12_005.mid;MP_B;./CalebRascon/MIDI_Grammar_TRADE_CharlieParker/;M_B;trade_Caleb_CharlieParker_rola4.mid
MP_A;./CharlieParker/MIDI_Grammar_SOLO_LEN12/;M_A;CharlieParker_Grammar_SOLO_LEN12_005.mid;MP_B;./CalebRascon/MIDI_Grammar_TRADE_CharlieParker/;M_B;trade_Caleb_CharlieParker_rola5.mid
MP_A;./CharlieParker/MIDI_Grammar_SOLO_LEN12/;M_A;CharlieParker_Grammar_SOLO_LEN12_003.mid;MP_B;./CalebRascon/MIDI_Grammar_TRADE_CharlieParker/;M_B;trade_Caleb_CharlieParker_rola3.mid
MP_A;./CharlieParker/MIDI_Grammar_SOLO_LEN12/;M_A;CharlieParker_Grammar_SOLO_LEN12_003.mid;MP_B;./CalebRascon/MIDI_Grammar_TRADE_CharlieParker/;M_B;trade_Caleb_CharlieParker_rola2.mid
MP_A;./CharlieParker/MIDI_Grammar_SOLO_LEN12/;M_A;CharlieParker_Grammar_SOLO_LEN

In [7]:
with open('./note_representation_bspline.json', 'w') as outfile:
    json.dump(note_representation_bspline, outfile)